# Setup

In [89]:
import os
import operator
import re
import json

from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.runnables import chain as as_runnable
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableConfig
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_core.messages import AnyMessage
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph, START

from typing import List, Optional
from typing import Annotated

from typing_extensions import TypedDict
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Set API key
api_key = os.environ["ANTHROPIC_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]

# Initialize models. We'll use a smaller LLM for most of the work, but a large LLM to distill conversations and writing the final report
fast_llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
slow_llm = ChatAnthropic(api_key=api_key, model="claude-3-5-sonnet-20240620")

In [3]:
example_topic = "AI agents and the potential economic, social, and environmental impacts."

# Draft Outline

In [4]:
# Define Outline Structured Output
class SubSection(BaseModel):
    title: str = Field(...,
                       title = "Subsection Title")
    
    description: str = Field(...,
                             title = "Description of subsection")

    @property
    def as_str(self) -> str:
        return f"## {self.title}\n\n{self.description}".strip()

class Section(BaseModel):
    title: str = Field(...,
                       title = "Section Title")
    
    description: str = Field(...,
                             title = "Section description")
    
    subsections: Optional[List[SubSection]] = Field(default = None,
                                                    title = "List of Subsections",
                                                    description = "A list of subsections under this section if there are any.")

    references: List[str] = Field(default_factory=list)
    
    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            f"## {subsection.title}\n\n{subsection.description}"
            for subsection in self.subsections or []
        )
        citations = "\n".join([f" [{i}] {cit}" for i, cit in enumerate(self.references)])
        return f"# {self.title}\n\n{self.description}\n\n{subsections}".strip() + f"\n\n{citations}".strip()

class Outline(BaseModel):    
    sections: List[Section] = Field(default_factory=list,
                                    title = "Sections", 
                                    description = "List of the sections in the outline")
    
    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"{sections}".strip()  

In [5]:
draft_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a wikipedia article writer, please never break character.
            Please create an outline for a wikipedia article based on the following topic:
            """
        ),
        (
            "user", "{topic}"
        ),
    ]
)

draft_outline_chain = draft_outline_prompt | fast_llm.with_structured_output(Outline)

In [6]:
draft_outline_example = draft_outline_chain.invoke({"topic": example_topic})

In [7]:
draft_outline_example.sections

[Section(title='Introduction', description='An overview of AI agents, their definitions, and relevance in contemporary society.', subsections=None, references=[]),
 Section(title='Economic Impacts', description='An exploration of how AI agents are transforming various economic sectors.', subsections=None, references=[]),
 Section(title='Social Impacts', description='Analysis of the social implications of AI agents on employment, privacy, and human interaction.', subsections=None, references=[]),
 Section(title='Environmental Impacts', description='Discussion on the environmental effects of AI technology, including resource consumption and potential benefits.', subsections=None, references=[]),
 Section(title='Ethical Considerations', description='An examination of the ethical dilemmas posed by AI agents, including bias, accountability, and transparency.', subsections=None, references=[]),
 Section(title='Future Prospects', description='A look at the future developments in AI agents and

# Full Research Path

## Survey Related Articles

In [8]:
# Generate related topics

class RelatedTopics(BaseModel):
    topics: List[str] = Field(
        description="List of related topics to help in generating perspectives.",
    )

related_topics_prompt = ChatPromptTemplate.from_template(
"""I'm writing a Wikipedia page for the topic mentioned below. 
Please identify and recommend some related subjects that might be interesting. 
I'm looking for related subjects that provide insights into interesting aspects commonly associated with this topic.

Feel free to list things that are only tangentially related

Please list as many subjects as you can.

Topic of interest: {topic}

make sure to call the RelatedTopics function.
"""
)

related_topics_chain = related_topics_prompt | fast_llm.with_structured_output(
    RelatedTopics
)

In [9]:
related_topics = await related_topics_chain.ainvoke({"topic": example_topic})

In [10]:
related_topics.topics

['AI agents',
 'economic impacts of AI',
 'social impacts of AI',
 'environmental impacts of AI',
 'automation',
 'machine learning',
 'robotics',
 'digital economy',
 'job displacement',
 'AI ethics',
 'sustainability',
 'smart cities',
 'data privacy',
 'algorithmic bias',
 'technological unemployment',
 'AI in healthcare',
 'AI in education',
 'AI governance',
 'future of work',
 'AI in agriculture',
 'AI and climate change']

In [39]:
def format_name(name):
    return re.sub(r'[^a-zA-Z0-9_-]', '', name)

class Perspective(BaseModel):
    name: str = Field(
        description="Name of the persona.", pattern=r"^[a-zA-Z0-9_-]{1,64}$"
    )
    role: str = Field(
        description="Role of the editor in the context of the topic.",
    )
    description: str = Field(
        description="Description of the editor's focus, concerns, and motives.",
    )

    @property
    def as_str(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nDescription: {self.description}\n"

class Editors(BaseModel):
    perspectives: List[Perspective] = Field(
        description="Comprehensive list of personas with their roles and descriptions.",
        # Add a pydantic validation/restriction to be at most M editors
    )

def update_references(references, new_references):
    if not references:
        references = {}
    references.update(new_references)
    return references


def update_perspective(perspective, new_perspective):
    # Can only set at the outset
    if not perspective:
        return new_perspective
    return perspective

class InterviewState(TypedDict):
    topic: str
    messages: Annotated[List[AnyMessage], operator.add] 
    references: Annotated[Optional[dict], update_references] = {}
    current_perspective: Annotated[Optional[Perspective], update_perspective]

# Define State for Article Generator
class ArticleGeneratorState(TypedDict):
    topic: Annotated[str, Field(description="The main topic of the article, set by the user at the beginning")]
    outline: Annotated[Outline, Field(default_factory=list, description="A running outline of the article")]
    perspectives: Annotated[Perspectives, Field(description = "List of all perspectives of editors in interview graph")]
    interview_results: Annotated[List[InterviewState], Field(default_factory=list, description="States of the interviews between the ")]
    article: Annotated[str, Field(description="Final article")]

In [29]:
# Get related articles with wikipedia retriever
wikipedia_retriever = WikipediaRetriever(load_all_available_meta=True, top_k_results=1)

def format_articles(unformattedArticles: List[str]) -> List[str]:
    formatted_articles = []
    for doc in unformattedArticles:
        article = doc[0]
        if isinstance(article, BaseException):
            continue
        # formatted_article = "\n\n" + "Title: " + article.metadata["title"] + "\n" + "\nCategories: " + ", ".join(article.metadata["categories"])
        formatted_article = "\n\n" + "Title: " + article.metadata["title"] + "\n" + "Summary:\n" + article.metadata["summary"]
        formatted_articles.append(formatted_article)
    return formatted_articles

@as_runnable
async def survey_related_articles(state: ArticleGeneratorState) -> List[str]:
    topic = state["topic"]
    related_topics = await related_topics_chain.ainvoke({"topic": topic})
    related_articles = await wikipedia_retriever.abatch(related_topics.topics, return_exceptions=True)
    return format_articles(related_articles)

In [30]:
related_articles = await survey_related_articles.ainvoke(ArticleGeneratorState(topic = example_topic))

In [31]:
related_articles

['\n\nTitle: Artificial intelligence\nSummary:\nArtificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nSome high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often 

## Generate Perspectives

In [32]:
gen_editors_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You need to select a diverse (and distinct) group of wikipedia editors who will 
            work together to create a comprehensive article on the given topic.
            Each of them represents a different perspective, role, or persona related to this topic.\
            You can use the provided related Wikipedia pages of related topics for inspiration. 
            For each writer, add a description of what they will focus on.

            Make sure to call the Persona function.

            Wiki page outlines of related topics for inspiration:
            {example_articles}""",
        ),
        ("user", "Topic of interest: {topic}"),
    ])

gen_editors_chain = gen_perspectives_prompt | fast_llm.with_structured_output(Editors)

In [33]:
@as_runnable
async def gen_editors(state: ArticleGeneratorState, related_articles: List[str]) -> Editors:
    topic = state["topic"]
    # Create default persona
    default_perspective = Perspective(name = "Default Perspective", 
                                      role = "Basic fact writer",
                                      description = "Basic fact writer focusing on broadly covering the basic facts about the topic.")
    # Generate editors
    editors = await gen_editors_chain.ainvoke({"example_articles": related_articles, "topic": topic})
    # Add default writer to beginning of list
    editors.perspectives.insert(0, default_perspective)
    return editors

In [36]:
example_editors = await gen_editors.ainvoke(ArticleGeneratorState(topic = example_topic), related_articles = related_articles)

In [37]:
example_editors.perspectives

[Perspective(name='Default Perspective', role='Basic fact writer', description='Basic fact writer focusing on broadly covering the basic facts about the topic.'),
 Perspective(name='Economic Analyst', role='Economic Impact Specialist', description='This editor will focus on analyzing the economic impacts of AI agents, including job displacement, productivity gains, and the potential for economic inequality. They will present data-driven insights on how AI can transform industries and the labor market.'),
 Perspective(name='Social Scientist', role='Social Impact Researcher', description='This editor will examine the social implications of AI agents, including changes in human interactions, ethical considerations, and societal norms. They will explore how AI could shape human behavior, relationships, and potential biases in AI systems.'),
 Perspective(name='Environmental Scientist', role='Sustainability Expert', description='This editor will investigate the environmental impacts of AI ag

## Research Interview Between Editors and Subject Matter Expert

### Persona Ask Question

In [43]:
gen_question_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an experienced Wikipedia writer and want to edit a specific page. \
Besides your identity as a Wikipedia writer, you have a specific focus when researching the topic. \
Now, you are chatting with an expert to get information. Ask good questions to get more useful information.

When you have no more questions to ask, say "Thank you so much for your help!" to end the conversation.\
Please only ask one question at a time and don't ask what you have asked before.\
Your questions should be related to the topic you want to write.
Be comprehensive and curious, gaining as much unique insight from the expert as possible.\

Stay true to your specific perspective:

{perspective}""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

def label_message_with_editor_name(ai_message: AIMessage, name: str):
    ai_message.name = format_name(name)
    return ai_message


def set_current_perspective(state: InterviewState, name: str):
    '''
    Set up the messages for the current writer.
    To do this we convert all messages in the history
    that are not from the current writer to a HumanMessage,
    so the current writer views the other writers' and experts'
    messages as HumanMessages.
    '''
    converted = []
    name = format_name(name)
    for message in state["messages"]:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    return {"messages": converted}

In [84]:
@as_runnable
async def gen_question(state: InterviewState):
    current_perspective = state["current_perspective"]
    name = format_name(current_perspective.name)
    question_chain = (
        RunnableLambda(set_current_perspective).bind(name=name)
        | gen_question_prompt.partial(perspective=current_perspective.as_str)
        | fast_llm
        | RunnableLambda(label_message_with_editor_name).bind(name=name)
    )
    question = await question_chain.ainvoke(state)
    # state = {**state, "messages": state["messages"] + [question]}
    return {"messages": [question]}

In [65]:
messages = [
    HumanMessage(f"So you said you were writing an article on {example_topic}?")
]
question = await gen_question.ainvoke(
    {
        "current_perspective": example_editors.perspectives[0],
        "messages": messages,
    }
)

'So you said you were writing an article on AI agents and the potential economic, social, and environmental impacts.?'

In [66]:
question["messages"]

[HumanMessage(content='So you said you were writing an article on AI agents and the potential economic, social, and environmental impacts.?'),
 AIMessage(content='Can you provide an overview of the main economic benefits and challenges associated with the implementation of AI agents in various industries?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 189, 'total_tokens': 212}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None}, name='DefaultPerspective', id='run-d088b3e5-720f-44cd-928c-c36f5e0158cb-0', usage_metadata={'input_tokens': 189, 'output_tokens': 23, 'total_tokens': 212})]

### Subject Matter Expert Answer Question

In [67]:
class Queries(BaseModel):
    queries: List[str] = Field(
        description="Comprehensive list of search engine queries to answer the user's questions.",
    )

gen_queries_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful research assistant. Query the search engine to answer the user's questions.",
        ),
        ("human", "{question}"),
    ]
)
gen_queries_chain = gen_queries_prompt | fast_llm.with_structured_output(Queries, include_raw=True)

In [68]:
queries = await gen_queries_chain.ainvoke(
    {"question": question["messages"][1].content}
)
queries["parsed"].queries

['economic benefits of AI agents in industries',
 'challenges of implementing AI agents in industries',
 'impact of AI on job market and economy',
 'AI agents in healthcare economic benefits',
 'AI in manufacturing economic impact',
 'AI in finance benefits and challenges',
 'AI agents in retail economic effects']

In [69]:
queries = await gen_queries_chain.ainvoke(
    {"question": [HumanMessage(content=question["messages"][1].content)]}
)
queries["parsed"].queries

['economic benefits of AI agents in industries',
 'challenges of AI implementation in industries',
 'impact of AI on productivity and efficiency',
 'cost savings from AI agents in business',
 'job displacement due to AI in various sectors',
 'AI in healthcare economic benefits and challenges',
 'AI in manufacturing economic impact',
 'AI in finance benefits and risks',
 'AI in retail economic advantages and challenges']

In [70]:
def get_last_message(state: InterviewState) -> str:
    if state["messages"]:
        return state["messages"][-1].content
    return ""

'''
# Tavily is typically a better search engine, but your free queries are limited
search_engine = TavilySearchResults(max_results=4)

@tool
async def search_engine(query: str):
    """Search engine to the internet."""
    results = tavily_search.invoke(query)
    return [{"content": r["content"], "url": r["url"]} for r in results]
'''

# DDG
search_engine = DuckDuckGoSearchAPIWrapper()


@tool
async def search_engine(query: str):
    """Search engine to the internet."""
    results = DuckDuckGoSearchAPIWrapper()._ddgs_text(query)
    return [{"content": r["body"], "url": r["href"]} for r in results]

async def search_and_format(queries):
    results = await search_engine.abatch(queries, return_exceptions=True)
    formatted_results = {}
    for result_list in results:
        if not isinstance(result_list, Exception):
            for result in result_list:
                formatted_results.update({result["url"]: result["content"]})
    return formatted_results

In [71]:
class AnswerWithCitations(BaseModel):
    answer: str = Field(
        description="Comprehensive answer to the user's question with citations.",
    )
    cited_urls: List[str] = Field(
        description="List of urls cited in the answer.",
    )

    @property
    def as_str(self) -> str:
        return f"{self.answer}\n\nCitations:\n\n" + "\n".join(
            f"[{i+1}]: {url}" for i, url in enumerate(self.cited_urls)
        )


gen_answer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert on the follwoing topic: {topic}. You are chatting with a Wikipedia writer who wants\
 to write a Wikipedia page on the topic you know. You have gathered the related information and will now use the information to form a response.

Make your response as informative as possible and make sure every sentence is supported by the gathered information.
Each response must be backed up by a citation from a reliable source, formatted as a footnote, reproducing the URLS after your response.

Here is the information you've gathered:
{search_results}
""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

Start: Test Code

In [90]:
# generate queries
last_message = "What are AI Agents?"
queries = await gen_queries_chain.ainvoke({"question": last_message})

# search and filter
search_results = await search_and_format(queries["parsed"].queries)
truncated_search_results = json.dumps(search_results)[:15000]

gen_answer_chain = gen_answer_prompt | fast_llm.with_structured_output(AnswerWithCitations, include_raw=True).with_config(run_name="GenerateAnswer")
answer = await gen_answer_chain.ainvoke({"topic": example_topic, "search_results": truncated_search_results})
cited_urls = set(answer["parsed"].cited_urls)

In [91]:
results_xt = await search_engine.abatch(queries["parsed"].queries, return_exceptions=True)

In [92]:
results_xt

[duckduckgo_search.exceptions.DuckDuckGoSearchException("https://duckduckgo.com/ return None. params=None content=None data={'q': 'What are AI Agents?'}"),
 duckduckgo_search.exceptions.DuckDuckGoSearchException("https://duckduckgo.com/ return None. params=None content=None data={'q': 'Definition of AI Agents'}"),
 duckduckgo_search.exceptions.DuckDuckGoSearchException("https://duckduckgo.com/ return None. params=None content=None data={'q': 'Types of AI Agents'}"),
 duckduckgo_search.exceptions.DuckDuckGoSearchException("https://duckduckgo.com/ return None. params=None content=None data={'q': 'Applications of AI Agents'}"),
 duckduckgo_search.exceptions.DuckDuckGoSearchException("https://duckduckgo.com/ return None. params=None content=None data={'q': 'How do AI Agents work?'}")]

End: Test Code

In [78]:
@as_runnable
async def gen_answer(
    state: InterviewState,
    name: str = "subject_matter_expert",
    max_str_length: int = 15000):

    name = format_name(name)
    # Set the expert as the current model
    current_state = set_current_perspective(state, name)
    #TODO: pass topic to this node
    topic = state["topic"]

    # generate queries
    last_message = get_last_message(state)
    queries = await gen_queries_chain.ainvoke({"question": last_message})

    # search and filter
    search_results = await search_and_format(queries["parsed"].queries)
    truncated_search_results = json.dumps(search_results)[:max_str_length]

    # answer the question
    gen_answer_chain = gen_answer_prompt | fast_llm.with_structured_output(AnswerWithCitations, include_raw=True).with_config(run_name="GenerateAnswer")
    answer = await gen_answer_chain.ainvoke({**current_state, "topic": topic, "search_results": truncated_search_results})
    cited_urls = set(answer["parsed"].cited_urls)

    # Format answer and get cited references
    formatted_answer = AIMessage(name=name, content=answer["parsed"].as_str)
    cited_references = {url: content for url, content in search_results.items() if url in cited_urls}

    # # Append the new message to the existing messages
    # updated_messages = state["messages"] + [formatted_answer]
    
    # # Merge the new references with the existing references
    # existing_references = state.get("references", {}) or {}  # Use empty dict if None
    # updated_references = {**existing_references, **cited_references}
    
    # Return the updated state
    return {"messages": [formatted_answer], "references": cited_references}

In [79]:
initial_state = {
    "topic": example_topic,
    "current_perspective": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"So you said you were writing an article on {example_topic}?",
            name="subject_matter_expert",
        )
    ],
}

example_answer = await gen_answer.ainvoke(
    initial_state
)

example_answer["messages"][-1].content

'AI agents, defined as autonomous systems capable of performing tasks without human intervention, have been increasingly integrated into various sectors, leading to profound economic, social, and environmental impacts. Economically, AI agents enhance productivity by automating repetitive tasks, which can lead to increased output and efficiency. For instance, McKinsey estimates that AI could contribute an additional $13 trillion to the global economy by 2030, significantly boosting growth in productivity and economic output (McKinsey Global Institute, 2018). However, this economic benefit comes with challenges, particularly in the labor market, where automation may displace jobs, especially in sectors like manufacturing and customer service, leading to potential unemployment and economic inequality (Bessen, 2019).  \n\nSocially, the integration of AI agents raises ethical concerns, such as bias in algorithmic decision-making and privacy issues related to data usage. AI systems can perpe

### Interview Graph

In [85]:
max_responses = 5

def end_or_pass_back_to_persona(state: InterviewState, name: str = "subject_matter_expert"):
    messages = state["messages"]
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and format_name(m.name) == format_name(name)]
    )
    if num_responses >= max_responses:
        return END
    last_question = messages[-2]
    if last_question.content.endswith("Thank you so much for your help!"):
        return END
    return "persona ask question"

In [86]:
# Build graph for interview
interview_graph = StateGraph(InterviewState)

# add nodes
interview_graph.add_node("persona ask question", gen_question)
interview_graph.add_node("subject matter expert answer question", gen_answer)

# add edges
interview_graph.add_edge(START, "persona ask question")
interview_graph.add_edge("persona ask question", "subject matter expert answer question")
interview_graph.add_conditional_edges("subject matter expert answer question", end_or_pass_back_to_persona)

# Compile graph
interview_graph = interview_graph.compile().with_config(run_name="Conduct Interviews")

In [87]:
final_result = None

initial_state = {
    "topic": example_topic,
    "current_perspective": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"So you said you were writing an article on {example_topic}?",
            name="subject_matter_expert",
        )
    ],
    "references": {},
}

async for chunk in interview_graph.astream(initial_state, stream_mode="values"):
    final_result = chunk

In [88]:
final_result

{'topic': 'AI agents and the potential economic, social, and environmental impacts.',
 'messages': [AIMessage(content='So you said you were writing an article on AI agents and the potential economic, social, and environmental impacts.?', name='subject_matter_expert'),
  AIMessage(content="Yes, I'm focusing on gathering comprehensive basic facts about AI agents and their various impacts. Can you explain the key economic impacts that AI agents are expected to have in different sectors?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 195, 'total_tokens': 230}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, name='DefaultPersona', id='run-16cde7df-4726-48f5-b050-ea158151890e-0', usage_metadata={'input_tokens': 195, 'output_tokens': 35, 'total_tokens': 230}),
  AIMessage(content="AI agents are expected to have profound economic impacts across various

# Build Final Graph

In [133]:
async def draft_outline(state: ArticleGeneratorState):
    topic, outline = state["topic"], state["outline"]
    outline = await draft_outline_chain.ainvoke({"topic": topic})
    return {
        **state,
        "outline": outline,
    }

# Conduct Research (Run interviews between all editors and subject_matter_expert
async def conduct_research(state: ArticleGeneratorState):
    topic = state["topic"]
    initial_interview_state = []
    for perspective in state["perspectives"].editors:
        initial_interview_states.append({
            "topic": topic,
            "messages": [
                AIMessage(
                    content=f"So you said you were writing an article on {topic}?",
                    name="subject_matter_expert",
                )
            ],
            "current_perspective": perspective
        })

    interview_results = await interview_graph.abatch(initial_interview_states)
    
    return {
        **state,
        "interview_results": interview_results,
    }

# Refine Outline
refine_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a Wikipedia writer. You have gathered information from experts and search engines. Now, you are refining the outline of the Wikipedia page. \
You need to make sure that the outline is comprehensive and specific. \
Topic you are writing about: {topic} 

Old outline:

{old_outline}""",
        ),
        (
            "user",
            "Refine the outline based on your conversations with subject-matter experts:\n\nConversations:\n\n{conversations}\n\nWrite the refined Wikipedia outline:",
        ),
    ]
)

# Using turbo preview since the context can get quite long
refine_outline_chain = refine_outline_prompt | slow_llm.with_structured_output(
    Outline
)

def format_conversation(interview_state):
    messages = interview_state["messages"]
    convo = "\n".join(f"{m.name}: {m.content}" for m in messages)
    return f'Conversation with {interview_state["editor"].name}\n\n' + convo


async def refine_outline(state: ArticleGeneratorState):
    convos = "\n\n".join(
        [
            format_conversation(interview_state)
            for interview_state in state["interview_results"]
        ]
    )

    updated_outline = await refine_outline_chain.ainvoke(
        {
            "topic": state["topic"],
            "old_outline": state["outline"].as_str,
            "conversations": convos,
        }
    )
    return {**state, "outline": updated_outline}

# Generate sections
gen_sections_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert Wikipedia writer. Complete your assigned WikiSection from the following outline:\n\n"
            "{outline}\n\nCite your sources, using the following references:\n\n{references}\n<Documents>",
        ),
        ("user", "Write the full WikiSection for the {section_title} section."),
    ]
)

gen_sections_chain = (
    gen_sections_prompt
    | slow_llm.with_structured_output(Section)
)

async def gen_sections(state: ArticleGeneratorState):
    topic, outline = state["topic"], state["outline"]
    outline.sections = await gen_sections_chain.abatch(
        [
            {
                "outline": outline.as_str,
                "section_title": section.title,
                "topic": topic,
            }
            for section in outline.sections
        ]
    )
    return {
        **state,
        "outline": outline,
    }

gen_article_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert Wikipedia author. Write the complete wiki article on {topic} using the following section drafts:\n\n"
            "{draft}\n\nStrictly follow Wikipedia format guidelines.",
        ),
        (
            "user",
            'Write the complete Wiki article using markdown format. Organize citations using footnotes like "[1]",'
            " avoiding duplicates in the footer. Include URLs in the footer.",
        ),
    ]
)

gen_article_chain = gen_article_prompt | slow_llm | StrOutputParser()


async def gen_article(state: ArticleGeneratorState):
    topic, outline = state["topic"], state["outline"]
    sections = outline.sections
    draft = "\n\n".join([section.as_str for section in sections])
    article = await gen_article_chain.ainvoke({"topic": topic, "draft": draft})
    return {
        **state,
        "article": article,
    }

In [135]:
# Build graph for Article Generator
article_gen_graph = StateGraph(ArticleGeneratorState)

# add nodes
article_gen_graph.add_node("draft outline", draft_outline)
article_gen_graph.add_node("survey related articles", survey_related_articles)
article_gen_graph.add_node("identify perspectives", gen_perspectives)
article_gen_graph.add_node("conduct research", conduct_research)
article_gen_graph.add_node("refine outline", refine_outline)
article_gen_graph.add_node("generate sections", gen_sections)
article_gen_graph.add_node("refine final article", gen_article)

# add edges
# Start with two concurrent flows
article_gen_graph.add_edge(START, "draft outline")
article_gen_graph.add_edge(START, "survey related articles")
# draft outline path
article_gen_graph.add_edge("draft outline", "refine outline")
# research path
article_gen_graph.add_edge("survey related articles", "identify perspectives")
article_gen_graph.add_edge("identify perspectives", "conduct research")
article_gen_graph.add_edge("conduct research", "refine outline")
# combined path after outline generation. Now to generate the article
article_gen_graph.add_edge("refine outline", "generate sections")
article_gen_graph.add_edge("generate sections", "refine final article")
article_gen_graph.add_edge("refine final article", END)

# compile graph
app = article_gen_graph.compile()

In [168]:
config = {"configurable": {"thread_id": "my-thread"}}
async for step in app.astream(
    {
        "topic": example_topic,
    },
    config,
):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name]))

draft outline
--  {'topic': 'AI agents and the potential economic, social, and environmental impacts.', 'outline': Outline(sections=[Section(title='Introduction', description='An overview of AI agents, their definition, and relevance in modern society.', subsections=None, references=[]), Section(title='Economic Impacts', description='A discussion of how AI agents are affecting economies worldwide.', subsections=[SubSection(title='Job Displacement', description='Analysis of the potential for AI agents to replace human jobs.'), SubSection(title='Productivity Gains', description='Examination of how AI agents can enhance productivity and efficiency in various sectors.'), SubSection(title='New Market Opportunities', description='Evaluation of new industries and job categories emerging from AI advancements.')], references=[]), Section(title='Social Impacts', description='An exploration of the social implications of AI agents on individuals and communities.', subsections=[SubSection(title='Ch

InvalidUpdateError: Expected dict, got ['\n\nTitle: Artificial intelligence\nSummary:\nArtificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nSome high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has fil